# Rethinking PII identification from Speech

The goal of this experiment is to evaluate the efficacy of various prompt-engineering techniques for PII identification. Previously, we employed an entity-aware ASR model—fine-tuned on Singlish speech and enhanced with an expanded tokenizer—to perform NER tagging directly on speech. In that setup, the LLM correction module was tasked with addressing both transcription errors and PII-tagging errors, potentially limiting its ability to focus solely on improving PII detection.

In this experiment, we will use the fine-tuned ASR model (trained on Singlish dialects) with its default tokenizer and delegate the entire PII tagging task to an LLM. This approach allows us to systematically compare different LLM prompting methods to determine which yields the best performance for our PII identification objectives.

## Step 0: Login Hugging Face CLI

## Step 1: Perform transcription with N-best using fine-tuned ASR

Skip this step if already transcribed

### Step 1.1: Download the model and tokenizer

In [1]:
import torch

if torch.cuda.is_available():
    device = "cuda"
    print("CUDA is available, using CUDA")
elif torch.backends.mps.is_available():
    device = "mps"
    print("MPS is available, using MPS")
else:
    device = "cpu"
    print("CUDA and MPS are not available, switching to CPU")

CUDA is available, using CUDA


In [2]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-small.en") # Using the default feature extractor and tokenizer
model = AutoModelForSpeechSeq2Seq.from_pretrained("f-azm17/whisper-small_en_seed_gretel_similar0.3").to(device)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

### Step 1.2: Load the dataset

For this example, we shall use the 150 dataset in the test set from `Audio_Files_for_testing`.

In [2]:
def retrieve_key(file: str) -> int:
    try:
        # 3 digit
        key = int(file[2:5])
    except ValueError:
        # 1 digit
        if file[3] == '.':
            key = int(file[2])
        else:
            key = int(file[2:4])
    return key

In [3]:
import os

audio_files = sorted(os.listdir("Audio_Files_for_testing"), key=retrieve_key)
audio_files = [f'Audio_Files_for_testing/{file}' for file in audio_files]
print(len(audio_files))

150


In [32]:
import pandas as pd

test_df = pd.DataFrame(data=audio_files, columns=['file_name'])
test_df.head()

,file_name
0,Audio_Files_for_testing/id1.wav
1,Audio_Files_for_testing/id2.wav
2,Audio_Files_for_testing/id3.wav
3,Audio_Files_for_testing/id4.wav
4,Audio_Files_for_testing/id5.wav


### Step 1.3: Transcribe

In [33]:
import librosa
from typing import List

def transcribe(audioPath: str, model: AutoModelForSpeechSeq2Seq, processor: AutoProcessor, device: str, best_n: int = 5) -> List[str]:
    """
    A function which transcribes the audio based on a given audio file path.
    Outputs the transcript along with the identified PII entities.
    
    Keyword arguments:
    audioPath (str) -- The path to the audio\n
    model (AutoModelForSpeechSeq2Seq) -- The ASR model\n
    processor (AutoProcessor) -- The processor, which contains the feature extractor and tokenizer.\n
    best_n (int) -- The best n number. By default, return the best transcription. 

    Return: The transcription along with the identified PII entities. (str)
    """
    waveform, sr = librosa.load(audioPath, sr=16000)
    inputs = processor(waveform, sampling_rate=sr, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            input_features=inputs["input_features"], 
            temperature=1.0,
            num_beams=best_n,
            num_return_sequences=best_n
        )
    transcriptions = processor.batch_decode(generated_ids, skip_special_tokens=True)
    return transcriptions

In [34]:
from tqdm import tqdm

for index, row in tqdm(test_df.iterrows(), desc="Transcribing and Identifying PII from test set...", total=len(test_df)):
    transcriptions = transcribe(row['file_name'], model, processor, device, 5)
    for i, transcription in enumerate(transcriptions):
        test_df.at[index, f'rank_{i+1}'] = transcription  

Transcribing and Identifying PII from test set...: 100%|██████████| 150/150 [03:17<00:00,  1.31s/it]


In [35]:
test_df.head()

,file_name,rank_1,rank_2,rank_3,rank_4,rank_5
0,Audio_Files_for_testing/id1.wav,the day before yesterday ram received another ...,"The day before yesterday, jason received anoth...","The day before yesterday, Ram received another...","The day before yesterday,ram received another ...",The day before yesterday RAM received another ...
1,Audio_Files_for_testing/id2.wav,um My date of birth is uh 2 september 19 92,mm my date of birth is uhm 2 september ninetee...,My date of birth is uh second september 19 92,"My date of birth is 2 september, 9092 H",mm My date of birth is uh second september ni...
2,Audio_Files_for_testing/id3.wav,hmm She handed over a crumpled piece of paper ...,she handed over a crumpled piece of paper ther...,She handed over a crumpled piece of paper Thi...,She handed over a crumpled piece of paper ther...,she handed over a crumpled piece of paper for...
3,Audio_Files_for_testing/id4.wav,uh and uh three of the other one ya,okay and uh three three of the other one yeah ...,I'll be picking it with another one and uh thr...,and uh uuh three three of the other ones yeah,uh and uh uh 3 3 of the other one yeah
4,Audio_Files_for_testing/id5.wav,uh Hong 's EMAIL is P X 1R z at 47 at ...,uhhh Hong s email is px 1 rzu 4 7 at yahoo...,Hong's email is P x1 rz'a 47 at yahoo.com,hongs email is P x 1 r z a 4 7 at yahoo dot com,hes saying hes still px one rz a four seven ...


In [36]:
test_df.to_csv('whisper-small_en_seed_gretel_similar0.3_no_tag_test_set_transcribed_n_best_5.csv', index=False)

### Step 1.4 (Optional): Load the transcribed files, if already transcribed

In [2]:
import pandas as pd

test_df = pd.read_csv('whisper-small_en_seed_gretel_similar0.3_no_tag_test_set_transcribed_n_best_5.csv')
test_df.head()

,file_name,rank_1,rank_2,rank_3,rank_4,rank_5
0,Audio_Files_for_testing/id1.wav,the day before yesterday ram received another ...,"The day before yesterday, jason received anoth...","The day before yesterday, Ram received another...","The day before yesterday,ram received another ...",The day before yesterday RAM received another ...
1,Audio_Files_for_testing/id2.wav,um My date of birth is uh 2 september 19 92,mm my date of birth is uhm 2 september ninetee...,My date of birth is uh second september 19 92,"My date of birth is 2 september, 9092 H",mm My date of birth is uh second september ni...
2,Audio_Files_for_testing/id3.wav,hmm She handed over a crumpled piece of paper ...,she handed over a crumpled piece of paper ther...,She handed over a crumpled piece of paper Thi...,She handed over a crumpled piece of paper ther...,she handed over a crumpled piece of paper for...
3,Audio_Files_for_testing/id4.wav,uh and uh three of the other one ya,okay and uh three three of the other one yeah ...,I'll be picking it with another one and uh thr...,and uh uuh three three of the other ones yeah,uh and uh uh 3 3 of the other one yeah
4,Audio_Files_for_testing/id5.wav,uh Hong 's EMAIL is P X 1R z at 47 at ...,uhhh Hong s email is px 1 rzu 4 7 at yahoo...,Hong's email is P x1 rz'a 47 at yahoo.com,hongs email is P x 1 r z a 4 7 at yahoo dot com,hes saying hes still px one rz a four seven ...


## Step 2: Perform ASR correction with N-best and in-context learning

We will now need to generate the best (corrected) transcription based on the 5-best list generated by the ASR. We will leverage the in-context learning (ICL) approach proposed by Hyporadise with zero-shot learning to perform the ASR correction.

The model used in the Hyporadise paper was GPT-3.5. As with the advancements to large language models and AI, the LLaMA-3.1-8b models have surpassed GPT-3.5 in many benchmarks, which can be seen in this link: https://www.vellum.ai/comparison/gpt-3-5-turbo-vs-llama-3-1-8b

### Step 2.1 Perform ASR correction with LLaMa

We shall use the Pipeline version to get the corrected ASR transcription, as the manual tokenizer + model approach seems to be simply outputting the input prompt.

#### One-shot in-context learning (As per the Hyporadise Paper)

In [3]:
target_domain = "conversational speech containing personal identifiable information"

one_shot_example = {
    "hypotheses": [
        "um My date of birth is uh 2 september  19 92",
        "mm my date of birth is uhm 2 september nineteen ninety-two",
        "My date of birth is uh second september 19 92",
        "My date of birth is 2 september,   9092 H",
        "mm My date of birth is  uh second september nineteen ninety-two"
    ],
    "expected_output": "um My date of birth is uh 2nd september 1992"
}

formatted_example_hypotheses = "\n".join([f"{i+1}: {hypothesis}" for i, hypothesis in enumerate(one_shot_example["hypotheses"])])

actual_hypotheses = []

questions = [
    "Are you familiar with speech recognition?",
    "Are you familiar with language model rescoring in ASR?",
    "Can you give a possible example on language model rescoring with 5-best hypotheses?",
    f"""
        Nice job, I will give you an example as a demonstration from {target_domain}. 
        The five best hypotheses list is:
        {formatted_example_hypotheses}
        
        I expect your output to be: {one_shot_example["expected_output"]}
        
        Following this example, can you report the true transcription from the following 5-best hypotheses?
    """
]

In [20]:
import gc
import transformers

# Load LLaMA 8B pipeline (uses bf16 for lower memory)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="cuda"
)

# Get the first row from test_df as a Series
some_row = test_df.iloc[22]

for i, question in enumerate(questions[3:]):
    # Build system prompt
    system_prompt = f"""
        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        {formatted_hypotheses}
        ---
        ### ANSWER ###
        ANSWER:
    """

    # Retrieve actual hypotheses for ASR correction
    actual_hypotheses = list(some_row[['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']])
    formatted_hypotheses = "\n".join([f"Hypothesis {i+1}: {hypothesis}" for i, hypothesis in enumerate(actual_hypotheses)])
    full_prompt = f"{system_prompt}\n### QUESTION ###\n{question}\n{formatted_hypotheses}"

    print("Prompt:\n")
    print(full_prompt)

    # Generate response using the LLaMA pipeline
    response = pipeline(
        full_prompt,
        max_new_tokens=256,  # Limit output length
        min_length=5,
        do_sample=False,  # Deterministic response
        temperature=0.0,  # Avoid randomness
        return_full_text=False,  # Prevents repeating the input prompt
        repetition_penalty=1.2
    )[0]["generated_text"]
    
    print("\nResponse:\n")
    print(response)

    # Free GPU memory after each run
    del full_prompt, response
    torch.cuda.empty_cache()
    gc.collect()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt:


        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        Hypothesis 1: the day before yesterday ram received another email from  R e m y at outlook.sg 
Hypothesis 2: The

In [20]:
import gc
import transformers

# Load LLaMA 8B pipeline (uses bf16 for lower memory)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="cuda"
)

# Get the first row from test_df as a Series
some_row = test_df.iloc[22]

for i, question in enumerate(questions[3:]):
    # Build system prompt
    system_prompt = f"""
        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        {formatted_hypotheses}
        ---
        ### ANSWER ###
        ANSWER:
    """

    # Retrieve actual hypotheses for ASR correction
    actual_hypotheses = list(some_row[['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']])
    formatted_hypotheses = "\n".join([f"Hypothesis {i+1}: {hypothesis}" for i, hypothesis in enumerate(actual_hypotheses)])
    full_prompt = f"{system_prompt}\n### QUESTION ###\n{question}\n{formatted_hypotheses}"

    print("Prompt:\n")
    print(full_prompt)

    # Generate response using the LLaMA pipeline
    response = pipeline(
        full_prompt,
        max_new_tokens=256,  # Limit output length
        min_length=5,
        do_sample=False,  # Deterministic response
        temperature=0.0,  # Avoid randomness
        return_full_text=False,  # Prevents repeating the input prompt
        repetition_penalty=1.2
    )[0]["generated_text"]
    
    print("\nResponse:\n")
    print(response)

    # Free GPU memory after each run
    del full_prompt, response
    torch.cuda.empty_cache()
    gc.collect()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt:


        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        Hypothesis 1: the day before yesterday ram received another email from  R e m y at outlook.sg 
Hypothesis 2: The

In [20]:
import gc
import transformers

# Load LLaMA 8B pipeline (uses bf16 for lower memory)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="cuda"
)

# Get the first row from test_df as a Series
some_row = test_df.iloc[22]

for i, question in enumerate(questions[3:]):
    # Build system prompt
    system_prompt = f"""
        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        {formatted_hypotheses}
        ---
        ### ANSWER ###
        ANSWER:
    """

    # Retrieve actual hypotheses for ASR correction
    actual_hypotheses = list(some_row[['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']])
    formatted_hypotheses = "\n".join([f"Hypothesis {i+1}: {hypothesis}" for i, hypothesis in enumerate(actual_hypotheses)])
    full_prompt = f"{system_prompt}\n### QUESTION ###\n{question}\n{formatted_hypotheses}"

    print("Prompt:\n")
    print(full_prompt)

    # Generate response using the LLaMA pipeline
    response = pipeline(
        full_prompt,
        max_new_tokens=256,  # Limit output length
        min_length=5,
        do_sample=False,  # Deterministic response
        temperature=0.0,  # Avoid randomness
        return_full_text=False,  # Prevents repeating the input prompt
        repetition_penalty=1.2
    )[0]["generated_text"]
    
    print("\nResponse:\n")
    print(response)

    # Free GPU memory after each run
    del full_prompt, response
    torch.cuda.empty_cache()
    gc.collect()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt:


        ### SYSTEM PROMPT ###
        You are selecting the best ASR transcription.
        
        RULES (Guidelines, But Selection is Mandatory):
        - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
        - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
        - Ignore capitalization differences.
        - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
        - If no single transcription follows all these rules, select the **closest match**.
        - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**
        
        ---
        ### QUESTION ###
        Select the best transcription from the following:
        Hypothesis 1: the day before yesterday ram received another email from  R e m y at outlook.sg 
Hypothesis 2: The

In [24]:
import gc
import torch
import transformers

# Load LLaMA 8B pipeline (using bf16 to save memory)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="cuda"
)

# Get a specific row from test_df
some_row = test_df.iloc[0]

for i, question in enumerate(questions[3:]):
    # Retrieve actual hypotheses for ASR correction
    actual_hypotheses = list(some_row[['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']])
    formatted_hypotheses = "\n".join([f"Hypothesis {i+1}: {hypothesis}" for i, hypothesis in enumerate(actual_hypotheses)])

    # Build system prompt
    system_prompt = f"""
    ### SYSTEM PROMPT ###
    You are selecting the best ASR transcription.

    RULES (Guidelines, But Selection is Mandatory):
    - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
    - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
    - Ignore capitalization differences.
    - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
    - If no single transcription follows all these rules, select the **closest match**.
    - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**

    ---
    """

    # Final formatted prompt
    full_prompt = f"{system_prompt}\n### QUESTION ###\n{question}\n{formatted_hypotheses}\n\n### ANSWER ###\nANSWER:"

    print("Prompt:\n")
    print(full_prompt)

    # Generate response using LLaMA pipeline
    response = pipeline(
        full_prompt,
        max_new_tokens=256,  # Limit output length
        min_length=5,
        do_sample=False,  # Deterministic response
        temperature=0.0,  # Avoid randomness
        return_full_text=False,  # Prevents repeating the input prompt
        repetition_penalty=1.2
    )[0]["generated_text"].strip()
    
    print("\nResponse:\n")
    print(response if response else "[ERROR: Blank Response]")

    # Free GPU memory after each run
    del full_prompt, response
    torch.cuda.empty_cache()
    gc.collect()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt:


    ### SYSTEM PROMPT ###
    You are selecting the best ASR transcription.

    RULES (Guidelines, But Selection is Mandatory):
    - Prefer numeric digits (e.g., '1234') over spelled-out numbers (e.g., 'one two three four') when both formats exist.
    - Prefer standard email formatting (e.g., 'john.doe@example.com') over verbalized formats (e.g., 'john dot doe at example dot com').
    - Ignore capitalization differences.
    - If multiple transcriptions are similar, prefer the most **frequent** format across all hypotheses.
    - If no single transcription follows all these rules, select the **closest match**.
    - **One answer MUST be chosen, even if no option is perfect. Do NOT leave the response blank.**

    ---
    
### QUESTION ###

        Nice job, I will give you an example as a demonstration from conversational speech containing personal identifiable information. 
        The five best hypotheses list is:
        1: um My date of birth is uh 2 september  19 92
